<a href="https://colab.research.google.com/github/hihiweiping/Weiwei/blob/main/ipick_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
search_ingr = str(input("設定主食材:"))

In [ ]:
filter_dislike = str(input("設定要避免的食物:"))

In [ ]:
while True:
  try:
    filter_like = int(input("設定食譜按讚數大於: "))
    break
  except ValueError:
      print("請輸入整數...")  
      continue

In [ ]:
while True:
  try:
    filter_ingr_count = int(input("設定食材數小於: "))
    break
  except ValueError:
      print("請輸入整數...")  
      continue

In [13]:
import time 
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
driver = Chrome("./chromedriver")
driver.get("https://icook.tw/")
driver.maximize_window()
e = driver.find_element_by_class_name("search-ingredient-placeholder")
e.send_keys(search_ingr)
time.sleep(1)
e.send_keys(Keys.ENTER)
time.sleep(1)
first_page = driver.current_url
first_page

In [14]:
import pandas as pd 
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import HTTPError


table = {
    "title":[],
    "ingr":[],
    "like":[],
    "href":[]
}

page = 1
while True: 
    url = first_page + "?page=" + str(page)
    req = Request(url)
    req.add_header("user-agent", "Mozilla/5.0")
    # print("頁數", page)
    page = page + 1

    try:
        response = urlopen(req)
    except HTTPError as e:
        if e.code == 400:
            print("最後一頁")
        else:
            # print("錯誤")
            df = pd.DataFrame.from_dict(table, orient='index')
            # df.to_csv("tofu.csv", encoding="utf-8", index=False)
            break
    html = BeautifulSoup(response)

    rs = html.find_all("li", class_="browse-recipe-item")
    for r in rs:
        title = r.find("h2", class_="browse-recipe-name")
        ingr = r.find("p", class_="browse-recipe-content-ingredient")
        like = r.find("li", class_="browse-recipe-meta-item")
        href = r.find("a", class_="browse-recipe-link")
        h = href["href"]
        # print(title.text.lstrip())
        # print(ingr.text.lstrip())
        # print(like.text.lstrip())
        # print("https://icook.tw" + href["href"])
        # print("-" * 30)
        try:
          table["title"].append(title.text)
          table["ingr"].append(ingr.text)
          table["like"].append(like.text)
          table["href"].append("https://icook.tw" + h)
        except:
          table["title"].append("0")
          table["ingr"].append("0")
          table["like"].append("0")
          table["href"].append("0")

In [15]:
df = pd.DataFrame.from_dict(table)
df = df.replace('\n','', regex=True)
df = df.replace('說讚','', regex=True)
df = df.replace('食材：','', regex=True)
df = df.replace(',','', regex=True)
df

,title,ingr,like,href
0,豆瓣醬炒豆腐,水豆腐、絞肉、蒜、蔥、辣椒、辣豆瓣醬、醬油、糖,14,https://icook.tw/recipes/366730
1,自製豆瓣醬高麗菜,高麗菜、蒜碎、花椒粒、酒、味霖、自製豆瓣醬、昆布柴魚高湯,2,https://icook.tw/recipes/365646
2,螞蟻上樹（辣豆瓣醬炒冬粉,絞肉、冬粉、鹽、薑、醬油、蒜、料理酒、蔥、白胡椒、辣豆瓣醬,75,https://icook.tw/recipes/360578
3,辣豆瓣炒雞肉vs桂冠輕沙拉醬,桂冠輕沙拉醬100克、雞胸肉250克、洋蔥、辣豆瓣醬、蒜末、薑末、青蔥,11,https://icook.tw/recipes/358514
4,鮮茄炒豆瓣醬,茄子、蒜末、醬油、油、辣豆瓣醬、蔥末、水、香油、太白粉、醬油膏,6,https://icook.tw/recipes/358262
...,...,...,...,...
71,燕麥冷豆腐搭豆瓣彩蔬淋醬,燕麥豆腐、薑切末、蒜切末、辣椒切圈、杏鮑菇切丁、綠花椰菜切小朵、...,88,https://icook.tw/recipes/50873
72,豆瓣醬燒五香豆干,豆瓣醬、蠔油、醬油、味霖、糖、傳貴五香豆干,1630,https://icook.tw/recipes/28327
73,涼拌小黃瓜佐豆瓣醬【莎拉游亂做廚房】,小黃瓜、豆瓣醬,116,https://icook.tw/recipes/28214
74,愛烹飪。豆瓣醬燒芋莖(芋槐),芋莖、蒜片、薑片、糖、醬油、辣豆瓣醬,78,https://icook.tw/recipes/25132


In [16]:
df['count_ingr'] = df['ingr'].str.count('、') +1
df['count_ingr']

0      8
1      7
2     10
3      7
4     10
      ..
71    13
72     6
73     2
74     6
75     9
Name: count_ingr, Length: 76, dtype: int64

In [17]:
df['like'] = df['like'].astype(int)
pop = df["like"] > filter_like
df_pop = df[pop]
df_pop

,title,ingr,like,href,count_ingr
34,螞蟻上樹之辣豆瓣醬🌶️,冬粉、豬絞肉、豆瓣醬、蒜頭（蒜末）、香油、醬油、薑（薑末）、醋、...,3975,https://icook.tw/recipes/176671,14
37,豆瓣醬蚵仔豆腐煲,豆腐、蚵仔、玉米粉、豆瓣醬、米酒、薑末、蒜末、蔥末、糖,1601,https://icook.tw/recipes/171255,9
63,豆瓣醬燒豆腐,板豆腐、蔥花、蒜末、辣椒末、豆瓣醬、【調味料】醬油、【調味料】糖...,3017,https://icook.tw/recipes/67266,8
72,豆瓣醬燒五香豆干,豆瓣醬、蠔油、醬油、味霖、糖、傳貴五香豆干,1630,https://icook.tw/recipes/28327,6


In [18]:
simple = df['count_ingr'] < filter_ingr_count
df_simple = df[simple]
df_simple

,title,ingr,like,href,count_ingr
0,豆瓣醬炒豆腐,水豆腐、絞肉、蒜、蔥、辣椒、辣豆瓣醬、醬油、糖,14,https://icook.tw/recipes/366730,8
1,自製豆瓣醬高麗菜,高麗菜、蒜碎、花椒粒、酒、味霖、自製豆瓣醬、昆布柴魚高湯,2,https://icook.tw/recipes/365646,7
3,辣豆瓣炒雞肉vs桂冠輕沙拉醬,桂冠輕沙拉醬100克、雞胸肉250克、洋蔥、辣豆瓣醬、蒜末、薑末、青蔥,11,https://icook.tw/recipes/358514,7
6,豆瓣醬燒杏鮑菇,杏鮑菇、豆瓣醬、蒜頭、薑片、油、醬油膏,22,https://icook.tw/recipes/350859,6
7,砂鍋豆瓣魚-豆油伯辣豆瓣醬,鮮魚、地瓜粉、蒜仁、白菜、蒜苗、橄欖油、豆油伯辣豆瓣醬、豆油伯麻...,7,https://icook.tw/recipes/344274,9
8,豆瓣醬炒冬粉,冬粉、豆瓣醬、醬油、肉絲、蔥、蒜頭、米酒,144,https://icook.tw/recipes/322749,7
9,豆瓣蒜醬牡蠣（飽滿不破的煮蚵法）,去殼牡蠣清肉、樹薯粉或地瓜粉、蒜、辣豆瓣醬/醬油/牡蠣水*,141,https://icook.tw/recipes/306781,4
10,豆瓣醬大黃瓜,大黃瓜、鹽、黃薑絲(可用生薑)、蒜碎、花椒粒、酒、味霖、郫縣豆瓣...,17,https://icook.tw/recipes/306607,9
11,辣豆瓣醬櫛瓜,綠櫛瓜、黃櫛瓜、辣豆瓣醬、豬絞肉、醬油、蒜頭、棕櫚糖或二砂、蔥花...,173,https://icook.tw/recipes/302931,9
12,豆瓣醬芹菜,芹菜、蒜碎、辣椒、花椒粒、酒、味霖、郫縣豆瓣醬、山西陳醋(可用任...,14,https://icook.tw/recipes/298900,9


In [19]:
pick = df.loc[df["ingr"].str.contains(filter_dislike)]
after_pickout=df.append(pick)
after_pickout.drop_duplicates(keep=False,inplace=True)
after_pickout

,title,ingr,like,href,count_ingr
0,豆瓣醬炒豆腐,水豆腐、絞肉、蒜、蔥、辣椒、辣豆瓣醬、醬油、糖,14,https://icook.tw/recipes/366730,8
2,螞蟻上樹（辣豆瓣醬炒冬粉,絞肉、冬粉、鹽、薑、醬油、蒜、料理酒、蔥、白胡椒、辣豆瓣醬,75,https://icook.tw/recipes/360578,10
3,辣豆瓣炒雞肉vs桂冠輕沙拉醬,桂冠輕沙拉醬100克、雞胸肉250克、洋蔥、辣豆瓣醬、蒜末、薑末、青蔥,11,https://icook.tw/recipes/358514,7
4,鮮茄炒豆瓣醬,茄子、蒜末、醬油、油、辣豆瓣醬、蔥末、水、香油、太白粉、醬油膏,6,https://icook.tw/recipes/358262,10
5,豆瓣醬炒梅花肉,梅花豬肉、洋蔥、辣豆瓣、蒜頭、味霖、朝天椒、雞蛋、蔥、白胡椒粉、...,20,https://icook.tw/recipes/357330,14
...,...,...,...,...,...
70,菇菇豆瓣醬,好菇道鴻喜菇、好菇道雪白菇、豬絞肉、不辣豆瓣醬、香菜、紅蘿蔔,252,https://icook.tw/recipes/51976,6
71,燕麥冷豆腐搭豆瓣彩蔬淋醬,燕麥豆腐、薑切末、蒜切末、辣椒切圈、杏鮑菇切丁、綠花椰菜切小朵、...,88,https://icook.tw/recipes/50873,13
72,豆瓣醬燒五香豆干,豆瓣醬、蠔油、醬油、味霖、糖、傳貴五香豆干,1630,https://icook.tw/recipes/28327,6
73,涼拌小黃瓜佐豆瓣醬【莎拉游亂做廚房】,小黃瓜、豆瓣醬,116,https://icook.tw/recipes/28214,2


In [20]:
df_pop_and_simple = df[(pop & simple)]
df_pop_and_simple

,title,ingr,like,href,count_ingr
37,豆瓣醬蚵仔豆腐煲,豆腐、蚵仔、玉米粉、豆瓣醬、米酒、薑末、蒜末、蔥末、糖,1601,https://icook.tw/recipes/171255,9
63,豆瓣醬燒豆腐,板豆腐、蔥花、蒜末、辣椒末、豆瓣醬、【調味料】醬油、【調味料】糖...,3017,https://icook.tw/recipes/67266,8
72,豆瓣醬燒五香豆干,豆瓣醬、蠔油、醬油、味霖、糖、傳貴五香豆干,1630,https://icook.tw/recipes/28327,6


In [21]:
all_filters_df = pd.merge(df_pop_and_simple, after_pickout, how='inner')
all_filters_df

,title,ingr,like,href,count_ingr
0,豆瓣醬蚵仔豆腐煲,豆腐、蚵仔、玉米粉、豆瓣醬、米酒、薑末、蒜末、蔥末、糖,1601,https://icook.tw/recipes/171255,9
1,豆瓣醬燒豆腐,板豆腐、蔥花、蒜末、辣椒末、豆瓣醬、【調味料】醬油、【調味料】糖...,3017,https://icook.tw/recipes/67266,8
2,豆瓣醬燒五香豆干,豆瓣醬、蠔油、醬油、味霖、糖、傳貴五香豆干,1630,https://icook.tw/recipes/28327,6
